In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip file_location

In [ ]:
import csv
import json
import math
import os
import shlex
import subprocess
import glob
import imageio
from optparse import OptionParser


def split_by_manifest(filename, manifest, vcodec="copy", acodec="copy",
                      extra="", **kwargs):
    """ Split video into segments based on the given manifest file.

    Arguments:
        filename (str)      - Location of the video.
        manifest (str)      - Location of the manifest file.
        vcodec (str)        - Controls the video codec for the ffmpeg video
                            output.
        acodec (str)        - Controls the audio codec for the ffmpeg video
                            output.
        extra (str)         - Extra options for ffmpeg.
    """
    if not os.path.exists(manifest):
        print("File does not exist: %s" % manifest)
        raise SystemExit

    with open(manifest) as manifest_file:
        manifest_type = manifest.split(".")[-1]
        if manifest_type == "json":
            config = json.load(manifest_file)
        elif manifest_type == "csv":
            config = csv.DictReader(manifest_file)
        else:
            print("Format not supported. File must be a csv or json file")
            raise SystemExit

        split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec,
                     "-acodec", acodec, "-y"] + shlex.split(extra)
        try:
            fileext = filename.split(".")[-1]
        except IndexError as e:
            raise IndexError("No . in filename. Error: " + str(e))
        for video_config in config:
            split_args = []
            try:
                split_start = video_config["start_time"]
                split_length = video_config.get("end_time", None)
                if not split_length:
                    split_length = video_config["length"]
                filebase = video_config["rename_to"]
                if fileext in filebase:
                    filebase = ".".join(filebase.split(".")[:-1])

                split_args += ["-ss", str(split_start), "-t",
                               str(split_length), filebase + "." + fileext]
                print("########################################################")
                print("About to run: " + " ".join(split_cmd + split_args))
                print("########################################################")
                subprocess.check_output(split_cmd + split_args)
            except KeyError as e:
                print("############# Incorrect format ##############")
                if manifest_type == "json":
                    print("The format of each json array should be:")
                    print("{start_time: <int>, length: <int>, rename_to: <string>}")
                elif manifest_type == "csv":
                    print("start_time,length,rename_to should be the first line ")
                    print("in the csv file.")
                print("#############################################")
                print(e)
                raise SystemExit


def get_video_length(filename):
    output = subprocess.check_output(("ffprobe", "-v", "error", "-show_entries", "format=duration", "-of",
                                      "default=noprint_wrappers=1:nokey=1", filename)).strip()
    video_length = int(float(output))
    print("Video length in seconds: " + str(video_length))

    return video_length


def ceildiv(a, b):
    return int(math.ceil(a / float(b)))


def split_by_seconds(filename, split_length, vcodec="copy", acodec="copy",
                     extra="", video_length=None, **kwargs):
    if split_length and split_length <= 0:
        print("Split length can't be 0")
        raise SystemExit

    if not video_length:
        video_length = get_video_length(filename)
    split_count = ceildiv(video_length, split_length)
    if split_count == 1:
        print("Video length is less then the target split length.")
        raise SystemExit

    split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec, "-acodec", acodec] + shlex.split(extra)
    try:
        filebase = ".".join(filename.split(".")[:-1])
        fileext = filename.split(".")[-1]
    except IndexError as e:
        raise IndexError("No . in filename. Error: " + str(e))
    for n in range(0, split_count):
        split_args = []
        if n == 0:
            split_start = 0
        else:
            split_start = split_length * n

        split_args += ["-ss", str(split_start), "-t", str(split_length),
                       filebase + "-" + str(n + 1) + "-of-" +
                       str(split_count) + "." + fileext]
        print("About to run: " + " ".join(split_cmd + split_args))
        subprocess.check_output(split_cmd + split_args)

def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

def mp4_to_avi(src_dir, dst_dir):
  reader = imageio.get_reader(src_dir)
  fps = reader.get_meta_data()['fps']
  writer = imageio.get_writer(dst_dir, fps=fps)

  for im in reader:
      writer.append_data(im[:, :, :])
  writer.close()

In [ ]:
!pip install imageio-ffmpeg

# SPLIT VIDEO
- Đổi path tới list video
- Muốn đổi thời gian cắt thì đổi biến split_time (đơn vị giây)

In [ ]:
path = '/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/*'
split_time = 10
for i in listPaths(path):
  if 'Split and Convert' not in i:
    split_by_seconds(i, split_time)

Video length in seconds: 60
About to run: ffmpeg -i /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/3902236034433.mp4 -vcodec copy -acodec copy -ss 0 -t 10 /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/3902236034433-1-of-6.mp4
About to run: ffmpeg -i /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/3902236034433.mp4 -vcodec copy -acodec copy -ss 10 -t 10 /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/3902236034433-2-of-6.mp4
About to run: ffmpeg -i /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/3902236034433.mp4 -vcodec copy -acodec copy -ss 20 -t 10 /content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning

# Convert MP4 to AVI
- Đổi input_path vào list video đã split ở trên.
- Đổi output_path vào folder chứa video.

In [ ]:
input_path = "/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/*"
output_path = "/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Deep Learning/Đồ án DS201 - Deep Learning/Data_ThuThap/Data_DN/Split and Convert/"

In [ ]:
index = 1
for src_dir in listPaths(input_path):
  if '-of-' in src_dir:
    dst_dir = output_path + 'Video' + str(index).zfill(5) + '.avi'
    mp4_to_avi(src_dir, dst_dir)
    index += 1